Выбор большой языковой модели

In [1]:
from llamba.chatmodels.ollama import OllamaModel
chatbot = OllamaModel(url="http://127.0.0.1:11434/", endpoint="api/generate", model="llama3", num_threads=1, check_connection_timeout=15, request_timeout=15) # опционально, параметр model может быть иным -- см. поддерживаемые модели Ollama
connection = chatbot.check_connection()
print(connection)

True


Подготовка данных

In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np

from llamba_library.bioage_model import BioAgeModel
from llamba_library.functions import get_shap_dict, get_top_shap
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_SHAP, FN_CHECKPOINT


#### Данные

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

data = pd.DataFrame(my_data, index=[0])


#### Модель
    
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(fn_model)
bioage_model = BioAgeModel(model=model)

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
shap_dict = get_shap_dict(FN_SHAP)
explainer = shap_dict['explainer']
feats = data.drop(['Age'], axis=1).columns.to_list()

top_n = 3 # количество признаков с наибольшим вкладом

sorted_values, sorted_data, sorted_feats = get_top_shap(n=top_n, data=data, feats=feats, shap_dict=shap_dict) 

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint H:\Lobachevsky\txai_omics_3\data\immuno\model.ckpt`
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


Составление запроса для экспертной системы 

In [3]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)
prompts = connector.generate_prompts(top_n=top_n, data=sorted_data, feats=sorted_feats, values=sorted_values) # n - количество признаков, внесших наибольший вклад
print(prompts)

['What is PDGFA? What does an increased level of PDGFA mean?', 'What is CD40LG? What does an increased level of CD40LG mean?', 'What is CXCL9? What does an increased level of CXCL9 mean?']


Передача экспертной системе данных 

In [4]:
res = connector.query_prompts()
print(res)

PDGFA: 1005.84429
CD40LG: 1853.847406
CXCL9: 2599.629474
PDGFA (Platelet-Derived Growth Factor A) is a protein that plays a role in cell growth, differentiation, and survival.

An increased level of PDGFA has been linked to various age-related conditions, including:

* Cellular senescence: As cells grow older, they may enter a state of permanent cell cycle arrest, characterized by an increase in PDGFA.
* Atherosclerosis: Elevated PDGFA levels have been found in the blood vessels of individuals with atherosclerosis, a condition that contributes to cardiovascular disease.
* Cancer development: Overexpression of PDGFA has been implicated in various types of cancer, including those affecting older adults.

In general, increased PDGFA levels may indicate cellular stress, inflammation, or aberrant cell growth, which can contribute to the development of age-related diseases.

CD40LG (also known as TNFSF5) is a protein encoded by the TNFSF5 gene. It is a member of the tumor necrosis factor rec